# DSAI HW2 : Addar & Subtractor Practice

In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/suchunying/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/Users/suchunying/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/suchunying/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/suchunying/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Parameters Config

## 設定參數
### 定義訓練集大小以及加入數字，分別設立正號跟負號個別的chars
### 給定RNN所需要的參數size

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
chars2 = '0123456789- '
chars3 = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)
ctable2 = CharacterTable(chars2)
ctable3 = CharacterTable(chars3)

In [6]:
ctable3.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# ----------------------------加法器---------------------------------

# Data Generation (adder)

### 產生訓練集以及測試集，question設定為方程式A+B，expected則為相對應的答案
### 並將資料轉化為one-hot representation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['647+3  ', '36+9   ', '634+1  ', '26+35  ', '61+96  '] ['650 ', '45  ', '635 ', '61  ', '157 ']


# Processing

### training size 18000 ， validation size 2000，test size 60000

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False  True False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False False False False False False False False
    True]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False False False False False  True False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False  True False False False False False False False
   False]
  [False False False False False False False False False False False
    True]
  [False False False False  True False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False False False Fal

# Build Model

### 設定RNN model
* activation為softmax
* loss function為categorical_crossentropy
* optimizer為adam
* metrics為accuracy

In [12]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [15]:
for iteration in range(100):
    #print()
    #print('-' * 50)
    #print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    #for i in range(1):
        #ind = np.random.randint(0, len(x_val))
        #rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        #preds = model.predict_classes(rowx, verbose=0)
        #q = ctable.decode(rowx[0])
        #correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        #print('Q', q[::-1] if REVERSE else q, end=' ')
        #print('T', correct, end=' ')
        #if correct == guess:
            #print(colors.ok + '☑' + colors.close, end=' ')
        #else:
            #print(colors.fail + '☒' + colors.close, end=' ')
        #print(guess)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 135us/step - loss: 0.8190 - acc: 0.6962 - val_loss: 0.8286 - val_acc: 0.6847
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 130us/step - loss: 0.7457 - acc: 0.7268 - val_loss: 0.7502 - val_acc: 0.7101
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 135us/step - loss: 0.6642 - acc: 0.7615 - val_loss: 0.6798 - val_acc: 0.7305
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 132us/step - loss: 0.5826 - acc: 0.8018 - val_loss: 0.5789 - val_acc: 0.7985
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 130us/step - loss: 0.5145 - acc: 0.8344 - val_loss: 0.5170 - val_acc: 0.8224
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000

18000/18000 [==============================] - 3s 141us/step - loss: 0.0188 - acc: 0.9983 - val_loss: 0.0509 - val_acc: 0.9861
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 147us/step - loss: 0.0199 - acc: 0.9978 - val_loss: 0.0501 - val_acc: 0.9866
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 146us/step - loss: 0.0370 - acc: 0.9906 - val_loss: 0.1206 - val_acc: 0.9591
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 144us/step - loss: 0.0660 - acc: 0.9799 - val_loss: 0.0540 - val_acc: 0.9844
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 140us/step - loss: 0.0279 - acc: 0.9948 - val_loss: 0.0525 - val_acc: 0.9856
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 140us/step - loss: 0

18000/18000 [==============================] - 3s 142us/step - loss: 0.1271 - acc: 0.9588 - val_loss: 0.0561 - val_acc: 0.9806
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 126us/step - loss: 0.0103 - acc: 0.9988 - val_loss: 0.0284 - val_acc: 0.9902
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 135us/step - loss: 0.0046 - acc: 1.0000 - val_loss: 0.0243 - val_acc: 0.9914
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 124us/step - loss: 0.0041 - acc: 0.9999 - val_loss: 0.0261 - val_acc: 0.9918
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 124us/step - loss: 0.0038 - acc: 0.9999 - val_loss: 0.0223 - val_acc: 0.9921
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 145us/step - loss: 0

# Testing

### 訓練好的模型拿來預測測試集，可以看到隨機抽取的預測結果基本上都是對的

In [42]:
print("MSG : Prediction")

MSG : Prediction


In [87]:
for i in range(10):
        ind = np.random.randint(0, len(test_x))
        rowx, rowy = test_x[np.array([ind])], test_y[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Q 84+752  T 836  ☑ 836 
Q 38+994  T 1032 ☑ 1032
Q 128+375 T 503  ☑ 503 
Q 66+646  T 712  ☑ 712 
Q 722+249 T 971  ☑ 971 
Q 35+569  T 604  ☑ 604 
Q 68+226  T 294  ☑ 294 
Q 777+712 T 1489 ☑ 1489
Q 88+971  T 1059 ☑ 1059
Q 33+810  T 843  ☑ 843 


# ------------------------------減法器---------------------------------

# Data Generation (subtractor)

### 生成減法的資料，把加號改成減號即可

In [92]:
questions1 = []
expected1 = []
seen1 = set()
print('Generating data...')
while len(questions1) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen1:
        continue
    seen1.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    #if REVERSE:
    #    query = query[::-1]
    questions1.append(query)
    expected1.append(ans)
print('Total addition questions:', len(questions1))

Generating data...
Total addition questions: 80000


In [93]:
print(questions1[:5], expected1[:5])

['7-77   ', '8-1    ', '952-21 ', '8-0    ', '166-57 '] ['-70 ', '7   ', '931 ', '8   ', '109 ']


# Processing

## 經測試後發現減法器在其他條件不變下需要更大的資料量才能提升準確率，
## 因此將訓練集數量增加到45000筆

In [94]:
print('Vectorization...')
x1 = np.zeros((len(questions1), MAXLEN, len(chars2)), dtype=np.bool)
y1 = np.zeros((len(expected1), DIGITS + 1, len(chars2)), dtype=np.bool)
for i, sentence in enumerate(questions1):
    x1[i] = ctable2.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected1):
    y1[i] = ctable2.encode(sentence, DIGITS + 1)

Vectorization...


In [95]:
indices1 = np.arange(len(y1))
np.random.shuffle(indices1)
x1 = x1[indices1]
y1 = y1[indices1]

# train_test_split
train_x1 = x1[:50000]
train_y1 = y1[:50000]
test_x1 = x1[50000:]
test_y1 = y1[50000:]

split_at = len(train_x1) - len(train_x1) // 10
(x_train1, x_val1) = train_x1[:split_at], train_x1[split_at:]
(y_train1, y_val1) = train_y1[:split_at], train_y1[split_at:]

print('Training Data:')
print(x_train1.shape)
print(y_train1.shape)

print('Validation Data:')
print(x_val1.shape)
print(y_val1.shape)

print('Testing Data:')
print(test_x1.shape)
print(test_y1.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Testing Data:
(30000, 7, 12)
(30000, 4, 12)


# Build Model

In [96]:
print('Build model...')

model1 = Sequential()
model1.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars2))))
model1.add(layers.RepeatVector(DIGITS + 1))

for _ in range(LAYERS):
    model1.add(RNN(HIDDEN_SIZE, return_sequences=True))

model1.add(layers.TimeDistributed(layers.Dense(len(chars2), activation='softmax')))
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model1.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [ ]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model1.fit(x_train1, y_train1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val1, y_val1))
    for i in range(10):
        ind = np.random.randint(0, len(x_val1))
        rowx, rowy = x_val1[np.array([ind])], y_val1[np.array([ind])]
        preds = model1.predict_classes(rowx, verbose=0)
        q = ctable2.decode(rowx[0])
        correct = ctable2.decode(rowy[0])
        guess = ctable2.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

# Testing

In [ ]:
preds = model1.predict_classes(test_x1)

count = 0
for i in range(len(preds)):
    
    q = ctable2.decode(test_x1[i])
    correct = ctable2.decode(test_y1[i])
    guess = ctable2.decode(preds[i], calc_argmax=False)
    
    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    
print(count/len(preds))

## 以測試集測試減法器準確率可以達到96.9%

In [99]:
print('the accuracy :')
print(count/len(preds))

the accuracy :
0.9694666666666667


# ---------------------------加法+減法------------------------------

### 合併加法跟減法的資料，訓練一個既可以學習加法也可以學習減法的運算模型
### 而因為資料變異較大，需要用更大量的資料來訓練，因此訓練集給了80000筆資料

In [119]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    questions.append(query)
    expected.append(ans)

print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [122]:
while len(questions) < 160000:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    #if REVERSE:
        #query = query[::-1]
    questions.append(query)
    expected.append(ans)

In [161]:
print(questions[:5],expected[:5])
print(questions[159995:],expected[159995:])

['5+1    ', '9+151  ', '152+314', '693+726', '8+72   '] ['6   ', '160 ', '466 ', '1419', '80  ']
['209-39 ', '858-69 ', '611-728', '71-372 ', '165-961'] ['170 ', '789 ', '-117', '-301', '-796']


In [142]:
print('Vectorization...')
x_ = np.zeros((len(questions), MAXLEN, len(chars3)), dtype=np.bool)
y_ = np.zeros((len(expected), DIGITS + 1, len(chars3)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x_[i] = ctable3.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y_[i] = ctable3.encode(sentence, DIGITS + 1)

Vectorization...


In [148]:
indices_ = np.arange(len(y_))
np.random.shuffle(indices_)
x_ = x_[indices_]
y_ = y_[indices_]

# train_test_split
train_x_ = np.concatenate((x_[:40000],x_[120000:]),axis=0)
train_y_ = np.concatenate((y_[:40000],y_[120000:]),axis=0)
test_x_ = x_[40001:120001]
test_y_ = y_[40001:120001]

split_at = len(train_x_) - len(train_x_) // 10
(x_train_, x_val_) = train_x_[:split_at], train_x_[split_at:]
(y_train_, y_val_) = train_y_[:split_at], train_y_[split_at:]

print('Training Data:')
print(x_train_.shape)
print(y_train_.shape)

print('Validation Data:')
print(x_val_.shape)
print(y_val_.shape)

print('Testing Data:')
print(test_x_.shape)
print(test_y_.shape)

Training Data:
(72000, 7, 13)
(72000, 4, 13)
Validation Data:
(8000, 7, 13)
(8000, 4, 13)
Testing Data:
(80000, 7, 13)
(80000, 4, 13)


In [151]:
print('Build model...')

model11 = Sequential()
model11.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars3))))
model11.add(layers.RepeatVector(DIGITS + 1))

for _ in range(LAYERS):
    model11.add(RNN(HIDDEN_SIZE, return_sequences=True))

model11.add(layers.TimeDistributed(layers.Dense(len(chars3), activation='softmax')))
model11.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model11.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 128)               72704     
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


## 訓練模型

In [ ]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model11.fit(x_train_, y_train_,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val_, y_val_))
    for i in range(10):
        ind = np.random.randint(0, len(x_val_))
        rowx, rowy = x_val_[np.array([ind])], y_val_[np.array([ind])]
        preds = model11.predict_classes(rowx, verbose=0)
        q = ctable3.decode(rowx[0])
        correct = ctable3.decode(rowy[0])
        guess = ctable3.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

## 測試後準確率可以達到96.8%

In [ ]:
preds_ = model11.predict_classes(test_x_)

count_ = 0
for i in range(len(preds_)):
    
    q = ctable3.decode(test_x_[i])
    correct = ctable3.decode(test_y_[i])
    guess = ctable3.decode(preds_[i], calc_argmax=False)
    
    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count_ += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    
print(count_/len(preds_))

In [155]:
print('the accuracy :')
print(count_/len(preds_))

the accuracy :
0.9689625
